# MLP con Early Stopping y Dropout (Core) / Elida Peña

**Descripción:**

En esta actividad, implementarás una red MLP más avanzada, utilizando técnicas como dropout y early stopping para evitar el sobreajuste. El dataset seleccionado es el «Heart Disease UCI» de Kaggle, donde se intentará predecir si un paciente tiene una enfermedad cardíaca o no basándose en varias características.

Enlace al dataset: https://www.kaggle.com/ronitf/heart-disease-uci



**Objetivo:**

El objetivo de esta actividad es entrenar una red MLP utilizando dropout para regularizar la red y early stopping para detener el entrenamiento cuando el rendimiento en el conjunto de validación deje de mejorar. Al final, se comparará el rendimiento de este modelo con el de un MLP básico.



**Instrucciones:**

1. Carga de datos:
  – Descarga el dataset «Heart Disease UCI» de Kaggle. Explora las características que están relacionadas con la edad, el sexo, la presión arterial y otras variables clínicas.
  – Verifica si hay valores nulos en el dataset y realiza el preprocesamiento necesario, como la imputación de valores faltantes y la normalización de las variables numéricas.

2. Exploración y preprocesamiento de datos:
  – Realiza una división del dataset en conjunto de entrenamiento y prueba. Asegúrate de que el dataset esté bien balanceado, verificando la distribución de la variable objetivo (enfermedad cardíaca sí/no).

3. Implementación de la MLP con Dropout:
  – Implementa una red MLP con dos capas ocultas utilizando Keras o cualquier biblioteca de deep learning.
  – Agrega capas de dropout después de cada capa oculta para reducir el riesgo de sobreajuste. Utiliza una tasa de dropout del 20-30%.
  – Implementa el early stopping para detener el entrenamiento cuando la precisión en el conjunto de validación deje de mejorar después de varias épocas (por ejemplo, paciencia de 5 épocas).

4. Entrenamiento del modelo:
  – Entrena el modelo con early stopping activado, utilizando el optimizador Adam y la función de pérdida binaria (binary_crossentropy). Entrena durante un máximo de 100 épocas.
  – Visualiza las curvas de aprendizaje para observar cómo evoluciona la pérdida y la precisión a lo largo del entrenamiento, especialmente con early stopping activado.

5. Evaluación del modelo:
  – Evalúa el modelo en el conjunto de prueba y compara las métricas de rendimiento, como precisión, recall, F1-score y matriz de confusión.
  – Discute cómo el uso de dropout y early stopping ha afectado el rendimiento en comparación con un MLP básico sin estas técnicas.

In [3]:
# Paso 1: Importación de librerías necesarias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [4]:
# Paso 1: Cargar el dataset
ruta_dataset = "../data/heart_disease_uci.csv"  # Ajusta esta ruta según corresponda
df = pd.read_csv(ruta_dataset)

# Mostrar las primeras filas
print("Primeras filas del dataset:")
print(df.head())

# Información general del dataset
print("\nInformación del dataset:")
print(df.info())

# Resumen estadístico
print("\nResumen estadístico:")
print(df.describe())



Primeras filas del dataset:
   id  age     sex    dataset               cp  trestbps   chol    fbs  \
0   1   63    Male  Cleveland   typical angina     145.0  233.0   True   
1   2   67    Male  Cleveland     asymptomatic     160.0  286.0  False   
2   3   67    Male  Cleveland     asymptomatic     120.0  229.0  False   
3   4   37    Male  Cleveland      non-anginal     130.0  250.0  False   
4   5   41  Female  Cleveland  atypical angina     130.0  204.0  False   

          restecg  thalch  exang  oldpeak        slope   ca  \
0  lv hypertrophy   150.0  False      2.3  downsloping  0.0   
1  lv hypertrophy   108.0   True      1.5         flat  3.0   
2  lv hypertrophy   129.0   True      2.6         flat  2.0   
3          normal   187.0  False      3.5  downsloping  0.0   
4  lv hypertrophy   172.0  False      1.4    upsloping  0.0   

                thal  num  
0       fixed defect    0  
1             normal    2  
2  reversable defect    1  
3             normal    0  
4       

In [5]:
# Verificar valores nulos
print("\n¿Existen valores nulos?")
print(df.isnull().sum())


¿Existen valores nulos?
id            0
age           0
sex           0
dataset       0
cp            0
trestbps     59
chol         30
fbs          90
restecg       2
thalch       55
exang        55
oldpeak      62
slope       309
ca          611
thal        486
num           0
dtype: int64


In [7]:
# Separar características (X) y etiquetas (y)
X = df.drop(columns=['num', 'id'])  # Eliminamos 'num' (etiqueta) y 'id' (irrelevante para el análisis)
y = (df['num'] > 0).astype(int)  # Convertir 'num' en binario: 0 = sin enfermedad, 1 = con enfermedad

# Paso 3: Manejo de valores nulos
# Imputar los valores nulos con la mediana para columnas con valores faltantes
X = X.fillna(X.median())

# Normalizar las características numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Verificar balance de clases
sns.countplot(x=y)
plt.title("Distribución de la variable objetivo (Clases)")
plt.show()

print("\nConjuntos de datos preparados:")
print(f"Tamaño del conjunto de entrenamiento: {X_train.shape}")
print(f"Tamaño del conjunto de prueba: {X_test.shape}")


TypeError: Cannot convert [['Male' 'Male' 'Male' ... 'Male' 'Male' 'Male']
 ['Cleveland' 'Cleveland' 'Cleveland' ... 'VA Long Beach' 'VA Long Beach'
  'VA Long Beach']
 ['typical angina' 'asymptomatic' 'asymptomatic' ... 'asymptomatic'
  'asymptomatic' 'atypical angina']
 ...
 [False True True ... False nan True]
 ['downsloping' 'flat' 'flat' ... nan nan nan]
 ['fixed defect' 'normal' 'reversable defect' ... 'fixed defect' nan nan]] to numeric

In [ ]:
# Paso 3: Implementación de la MLP con Dropout
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Primera capa oculta
    Dropout(0.3),  # Dropout con tasa del 30%
    Dense(32, activation='relu'),  # Segunda capa oculta
    Dropout(0.2),  # Dropout con tasa del 20%
    Dense(1, activation='sigmoid')  # Capa de salida para clasificación binaria
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Mostrar el resumen del modelo
model.summary()


In [8]:
# Paso 4: Entrenamiento con Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=32, 
                    callbacks=[early_stopping], verbose=1)

# Visualización de curvas de aprendizaje
plt.figure(figsize=(12, 6))

# Pérdida
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()

# Precisión
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Precisión durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()

plt.tight_layout()
plt.show()

# Paso 5: Evaluación del modelo
# Evaluar el modelo en el conjunto de prueba
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calcular métricas
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Sin enfermedad', 'Enfermedad'])
disp.plot(cmap=plt.cm.Blues)
plt.title('Matriz de confusión')
plt.show()

NameError: name 'model' is not defined